In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

#plotly.tools.set_credentials_file(username='slotbite', api_key='GYgRt3eXz6IdiIy3k7sO')
py.plotly.tools.set_credentials_file(username='cachulooo', api_key='srvpWbqoQ27WdGO3pArx')

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 20)

In [10]:
# Cargando datos
data = pd.read_csv("segmentacion_clientes.csv" , delimiter=";" , decimal="," , encoding='utf-8')
df=data

In [11]:
## Se eliminan preduct_id mal formados (-)
#df[df.PRODUCT_ID < 0] 
df = df[df.PRODUCT_ID > 0] 


## Convertimos las variables categoricas
#http://pbpython.com/categorical-encoding.html
pd.options.mode.chained_assignment = None
df["COMUNA"] = df["COMUNA"].astype('category')
df["TIPO_REPARTO"] = df["TIPO_REPARTO"].astype('category')
df["TI"] = df["TI"].astype('category')
df["TI2"] = df["TI2"].astype('category')

# luego la encodificamos 
df["TI"] = df["TI"].cat.codes
df["TI2"] = df["TI2"].cat.codes
df["COMUNA_ENC"] = df["COMUNA"].cat.codes 
df["TIPO_REPARTO"] = df["TIPO_REPARTO"].cat.codes 
  
df[:3]

,PRODUCT_ID,SUBSCRIPTION_ID,SUBSCRIBER_ID,COMUNA,TI,TI2,TIPO_REPARTO,QUEJAS,DAÑOS,CORTES,RECLAMOS,FACTURACION_PROMEDIO,SALDO_PROMEDIO,ROTACION_CARTERA,COMUNA_ENC
0,122551,122551,13185036,128 - San Esteban ...,0,0,0,0,0,0,0,18964.8333,33256.6667,1.753504,9
1,122581,122581,122581,128 - San Esteban ...,0,0,0,0,0,0,0,28445.7500,36605.0833,1.286793,9
2,109938,109938,3665521,145 - San Felipe ...,0,1,0,0,0,1,0,135045.4170,327823.5000,2.427488,22


In [12]:
## Escalado de la variables rotacion de cartera 
#https://stackoverflow.com/questions/50546065/scikit-learn-minmax-scaler-doesnt-scale
#from sklearn.preprocessing import MinMaxScaler 
#columna=['ROTACION_CARTERA'] 
#scaler = MinMaxScaler(feature_range=(0, 500000)) 
#df[columna] = scaler.fit_transform(df[columna])


## Seleccion de variables 

In [21]:
sample=df[['QUEJAS', 'DAÑOS',
            'CORTES', 'RECLAMOS','FACTURACION_PROMEDIO',
            'SALDO_PROMEDIO', 'ROTACION_CARTERA' ,
           'COMUNA_ENC' , 'TI' , 'TI2' ,'TIPO_REPARTO' ]].sample(frac=1, random_state=42)
sample.shape

(631293, 11)

In [22]:
#sample = sample[  (sample.QUEJAS==9)]   
sample = sample[(sample.QUEJAS==2)  ]
#sample = sample[(sample.QUEJAS==14) | (sample.QUEJAS==9)]

sample.shape[0]

1687

In [23]:
X = np.array(sample[['SALDO_PROMEDIO','ROTACION_CARTERA']] ) 
# Create a trace
trace = go.Scatter(
    x =  X[:, 0],  
    y =  X[:, 1],
    #mode = 'markers'  ,
    mode='markers+text' 
)  
layout = go.Layout(
    title='  '  ,
    plot_bgcolor = '#FFFFFF',
    paper_bgcolor = '#FFFFFF' , 
    xaxis=dict(
        title='saldo prom' 
    ),
    yaxis=dict(
        title='Rot cart' 
    ) 
)
data = [trace  ] 
fig = go.Figure(data = data, layout = layout)
# Plot and embed in ipython notebook!
py.iplot(fig ,   filename='scatter') 

## Silhouette  test
Determina el numero optimo de clusters

In [80]:
rango_vecinos=[]
num_clusters = []
silueta = []

X = np.array(sample )  
# Estandarización
#scaler = StandardScaler().fit(X)
#X = scaler.transform(X)

for n in   range(1 , 30) :  
    kmeans = KMeans(n_clusters=n+1 , init='k-means++' ).fit(X)
    num_clusters.append( n+1 )
    silueta.append( metrics.silhouette_score( X , kmeans.predict(X) ))  ## data y labels
  
    
bench_df = pd.DataFrame( { "num_clusters": num_clusters  , "silueta":silueta} ) 

In [81]:
bench_df.to_csv('bench.csv', sep=';' , index=False , decimal = ',')

In [82]:
# Create a trace
trace = go.Scatter(
    x = bench_df.num_clusters,  
    y = bench_df.silueta,     
    #mode = 'markers'  ,
    mode='markers+text',
    text=round(bench_df.num_clusters)
)  
layout = go.Layout(
    title='Bench kmenas Silueta vs N° Clusters '  ,
    plot_bgcolor = '#FFFFFF',
    paper_bgcolor = '#FFFFFF' , 
    xaxis=dict(
        title='N° Clusters' 
    ),
    yaxis=dict(
        title='Silueta' 
    ) 
)
data = [trace  ] 
fig = go.Figure(data = data, layout = layout)
# Plot and embed in ipython notebook!
py.iplot(fig ,   filename='minpts_bench')

 ## Clustering por KMEANS

In [99]:
## KMEANS
####################################################################
  
X = np.array(sample)

registros = sample.shape[0]

# Estandarización
#scaler = StandardScaler().fit(X)
#X = scaler.transform(X)

# Creando el modelo
k=8
kmeans = KMeans(n_clusters=k , init='k-means++' ).fit(X)
 
# Prediciendo los clusters
labels = kmeans.predict(X)
  
# Obteniendo los centroides
#C = kmeans.cluster_centers_

# Asignando los colores
#colores=['red','cyan','yellow','orange','purple' , 'red','cyan','yellow','orange','purple'] ## misma cantidad de colores que numero de k
#asignar=[]

#for row in labels:
#    asignar.append(colores[row])

In [100]:
# Create a trace
trace = go.Scatter(
    x = X[:, 5],   
    y = X[:, 6],   
    mode = 'markers', 
    marker=dict(color= labels   ,
                colorscale=[
                    [0.0               , 'rgb(255, 0, 102)'],
                    [0.0111111111111111, 'rgb(79, 77, 77)'], 
                    [0.1111111111111111, 'rgb(71, 51, 0)'], 
                    [0.1555555555555555, 'rgb(124, 118, 0)'],
                    [0.2222222222222222, 'rgb(171,217,233)'],  
                    [0.2555555555555555, 'rgb(69,117,180)'],  
                    [0.3333333333333333, 'rgb(255, 102, 204)'], 
                    [0.3555555555555555, 'rgb(137, 19, 173)'], 
                    [0.4444444444444444, 'rgb(165,0,38)'],  
                    [0.4555555555555555, 'rgb(102, 255, 51)'], 
                    [0.5555555555555556, 'rgb(215,48,39)'], 
                    [0.6666666666666666, 'rgb(255, 255, 0)'],  
                    [0.7777777777777778, 'rgb(244,109,67)'], 
                    [0.8888888888888888, 'rgb(116,173,209)'], 
                    [1.0               , 'rgb(49,54,149)']]  ) 
) 
layout = go.Layout(
    title='Segmentación de clientes KMEANS '+str(registros)+' Registros'+' '+str(k)+' Clusters' ,
    plot_bgcolor = '#FFFFFF',
    paper_bgcolor = '#FFFFFF', 
    xaxis=dict(
            title='SALDO_PROMEDIO' 
        ),
        yaxis=dict(
            title='ROTACION_CARTERA' 
        ) 
)
data = [trace] 
fig = go.Figure(data = data, layout = layout)
# Plot and embed in ipython notebook!
py.iplot(fig ,   filename='scatter_hdbscan')

In [85]:
## COORDENADAS PARALELAS 
data = [
    go.Parcoords(
        line = dict(color = labels  , colorscale=[
                    [0.0               , 'rgb(255, 0, 102)'],
                    [0.0111111111111111, 'rgb(79, 77, 77)'], 
                    [0.1111111111111111, 'rgb(71, 51, 0)'], 
                    [0.1555555555555555, 'rgb(124, 118, 0)'],
                    [0.2222222222222222, 'rgb(171,217,233)'],  
                    [0.2555555555555555, 'rgb(69,117,180)'],  
                    [0.3333333333333333, 'rgb(255, 102, 204)'], 
                    [0.3555555555555555, 'rgb(137, 19, 173)'], 
                    [0.4444444444444444, 'rgb(165,0,38)'],  
                    [0.4555555555555555, 'rgb(102, 255, 51)'], 
                    [0.5555555555555556, 'rgb(215,48,39)'], 
                    [0.6666666666666666, 'rgb(255, 255, 0)'],  
                    [0.7777777777777778, 'rgb(244,109,67)'], 
                    [0.8888888888888888, 'rgb(116,173,209)'], 
                    [1.0               , 'rgb(49,54,149)']]   ) ,  
        dimensions = list([
            dict(#range = [0,8], 
                label = 'QUEJAS' ,      values = X[:, 0]),
            dict( #range = [0,30],
                label = 'DAÑOS',        values = X[:, 1]),
            dict( #range = [0,8],
                label = 'CORTES',       values = X[:, 2]),
            dict( #range = [0,8],
                label = 'RECLAMOS',     values = X[:, 3]),
            dict( #range = [0,500000],
                label = 'FACTURACION_PROMEDIO', values = X[:, 4]),
            dict( #range = [0,500000],
                label = 'SALDO_PROMEDIO',   values = X[:, 5]),             
            dict( #range = [0,3],
                label = 'ROTACION_CARTERA', values = X[:, 6]),  
            dict( #range = [0,8],
                label = 'COMUNA',        values = X[:, 7]), 
            dict( #range = [0,8],
                label = 'TI',        values = X[:, 8]), 
            dict( #range = [0,8],
                label = 'TI2',        values = X[:, 9]), 
            dict( #range = [0,8],
                label = 'TIPO_REPARTO',        values = X[:, 10]),  
            dict( #range = [-1, n_clusters_-1 ],#   K CLUSTERS
                label = 'CLUSTER', values = labels) 
        ])
    )
]

layout = go.Layout(
    title='Segmentación de clientes KMEANS '+str(registros)+' Registros'+' '+str(k)+' Clusters' ,
    plot_bgcolor = '#FFFFFF',
    paper_bgcolor = '#FFFFFF'
) 
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'cpKMEANS'+str(registros))

In [77]:
np.unique(labels)

array([0, 1, 2, 3, 4])